In [ ]:
import simpy
import random
import numpy as np

In [ ]:
!pip install simpy

In [ ]:
class BikeStation:
  def __init__(self, env, numberOfBikes,name,infinite):
    self.env=env
    self.bikesQueue=simpy.Store(env, capacity=numberOfBikes) ## can hold an unlimited number of bikes, but for now holds numberOfBikes?
    self.name=name
    if infinite:
      for num in range(40000):
        self.bikesQueue.put("bike")
    else:
      for num in range(numberOfBikes):
        self.bikesQueue.put("bike")
  def pickUpBike(self):
    return self.bikesQueue.get() ##waits to get a bike if there aren't any available, need to return so that it stops waiting
  def returnBike(self):
    self.bikesQueue.put("bike")




def biker(stations,stationAProbabilities, env, meanRate,QMatrix,success, std, mu, stationProbabilities,avgWaitingTime):

  yield env.timeout(random.expovariate(meanRate))
  ##picking a station randomly
  stationChosen= np.random.choice(len(stationAProbabilities), p=stationProbabilities)
  stationName=list(stationAProbabilities.keys())[stationChosen]
  station=stations[stationName] ##now station is the station object
  beforePickingUp=env.now
  yield station.pickUpBike() ##waits until a bike is available
  afterPickingUp=env.now
  waitingTime=afterPickingUp-beforePickingUp
  avgWaitingTime.append(waitingTime)
  success[0]+=1
  ##choosing destination randomly
  startArray=QMatrix[stationName]
  ##now startArray is list of tuples with ending station and probabiities
  probabilities=[]
  for end,probability in startArray:
    probabilities.append(probability)

  destinatationStation=np.random.choice(len(startArray), p=probabilities) ##getting probability of going to station j starting at stationChosen index



  destination=startArray[destinatationStation][0]
  yield env.timeout(np.random.lognormal(mu,std)) ##random amount of time that the biker uses the bike

  if destination in stations:

    destinationObject=stations[destination]
    destinationObject.returnBike() ##Returning the bike to the destintations station





In [ ]:
def simulation(numberOfStations,bikesPerStation, numberOfRiders,meanRate,QMatrix,std,mu,stationAProbabilities):
  stations={}
  env=simpy.Environment()
  stationProbabilities=list(stationAProbabilities.values())
  infinite=False
  if(bikesPerStation==float('inf')):
    infinite=True
  for stationName,probability in stationAProbabilities.items():
    stations[stationName]=BikeStation(env,bikesPerStation,stationName,infinite)
  avgWaitingTime=[]
  success=[0]
  for num in range(numberOfRiders):
    env.process(biker(stations,stationAProbabilities, env, meanRate,QMatrix,success, std, mu, stationProbabilities,avgWaitingTime))
  env.run(until=60*24)  # Run the simulation for 24 hours (in minutes)
  sum=0
  count=0
  for num in avgWaitingTime:
    sum+=num
    count+=1
  avgWaiting=sum/count
  return success[0],avgWaiting  # Return the count of successful pickups



TASK 2

In [ ]:
import csv

# Corrected file path
csv_file_path = "/content/start_station_probs.csv"  # Using a raw string

# Initialize an empty list to store the second column data
stationAProbabilities={}
# Open the CSV file and read data

with open(csv_file_path, newline='') as csvfile:
    reader = csv.reader(csvfile)
    next(reader)
    for row in reader:
        if len(row) > 1:  # Make sure the row has at least two columns
            stationAProbabilities[row[0]]=float(row[1])  # Append data from the second column
print(stationAProbabilities)


{'South Waterfront Walkway - Sinatra Dr & 1 St': 0.04467944609528249, 'Grove St PATH': 0.04350443784109227, 'Hoboken Terminal - Hudson St & Hudson Pl': 0.0336285420186836, 'Hoboken Terminal - River St & Hudson Pl': 0.02983161451960613, 'Newport Pkwy': 0.027034900658393056, 'City Hall - Washington St & 1 St': 0.02635514381712599, 'Newport PATH': 0.025471459923478802, '12 St & Sinatra Dr N': 0.024422692225523897, 'Hoboken Ave at Monmouth St': 0.023684670512148225, 'Marin Light Rail': 0.023529297519858612, 'Hamilton Park': 0.022985492046844958, '14 St Ferry - 14 St & Shipyard Ln': 0.021616267552292723, 'Liberty Light Rail': 0.02025675386975859, 'Columbus Dr at Exchange Pl': 0.019285672667948495, 'Harborside': 0.018664180698790032, '11 St & Washington St': 0.017644545436889432, 'Washington St': 0.017265823768183496, 'Sip Ave': 0.017032764279749073, 'Hudson St & 4 St': 0.016955077783604264, '8 St & Washington St': 0.015750937093359747, 'Madison St & 1 St': 0.015488745168871021, 'City Hall':

In [ ]:
# Corrected file path
csv_file_path = "/content/trip_stats.csv"  # Using a raw string

# Initialize an empty list to store the second column data
tripStats= []

# Open the CSV file and read data

with open(csv_file_path, newline='') as csvfile:
    reader = csv.reader(csvfile)
    next(reader)
    for row in reader:
        if len(row) > 1:  # Make sure the row has at least two columns
            tripStats.append(tuple(row))  # Append data from the second column



In [ ]:
probabilityOfAB={}
total=0
for start,end,value,mean,std in tripStats:
  total+=int(value)
for start,end,value,mean,std in tripStats:
  probabilityOfAB[(start,end)]=int(value)/total



In [ ]:
ProbabilityOfBGivenA={}
Q={}
for key,probability in probabilityOfAB.items():
  start,end=key
  probabilityOfA=stationAProbabilities[start]
  ProbabilityOfBGivenA[(end,start)]=(float(probability)/probabilityOfA)
  if start not in Q:
    Q[start]=[]
  Q[start].append((end,ProbabilityOfBGivenA[end,start]))


In [ ]:
ProbabilityOfBGivenA[('Willow Ave & 12 St','11 St & Washington St')]

0.017611447440836543

In [ ]:
num_stations = len(stationAProbabilities)
bikesPerStation=10
num_riders = 3500
lambda_rate = 2.38
q_matrix = Q
std=.619
mu = 2.78



successful_rides,waiting = simulation(num_stations,bikesPerStation, num_riders,lambda_rate,q_matrix,std,mu,stationAProbabilities)
print(f"Total successful rides: {successful_rides}")
print(f"Waiting : {waiting}")


Total successful rides: 3475
Waiting : 32.50022717723433


In [ ]:
successfullRides=[]

for num in range(5):
  num_stations = len(stationAProbabilities)
  bikesPerStation=10
  num_riders = 3500
  lambda_rate = 2.38
  q_matrix = Q
  std=.619
  mu = 2.78



  successful_rides,waiting = simulation(num_stations,bikesPerStation, num_riders,lambda_rate,q_matrix,std,mu,stationAProbabilities)
  successfullRides.append(successful_rides)
print(f"Total successful rides: {successful_rides}")
print(f"Waiting : {waiting}")

Total successful rides: 3473
Waiting : 34.48093540258836


In [ ]:
import numpy as np
from scipy.stats import norm


successful_rides=sum(successfullRides)/5
total_riders=num_riders

# Calculate the sample proportion
p =successful_rides / total_riders

# Calculate the standard error (SE)
se = np.sqrt(p * (1 - p) / total_riders)


z_score = norm.ppf(0.95)

margin_of_error=z_score * se

# Calculate the confidence interval
ci_lower = p - margin_of_error
ci_upper = p + margin_of_error

# Print the results
print(f"90% Confidence Interval for the proportion of successful rides: ({ci_lower:.4f}, {ci_upper:.4f})")


90% Confidence Interval for the proportion of successful rides: (0.9784, 0.9857)


TASK 3

In [ ]:
class BikeStationTask3:
  def __init__(self, env, numberOfBikes,name,infinite):
    self.env=env
    self.bikesQueue=simpy.Store(env, capacity=numberOfBikes) ## can hold an unlimited number of bikes, but for now holds numberOfBikes?
    self.name=name
    self.visitors=0
    if infinite:
      for num in range(40000):
        self.bikesQueue.put("bike")
    else:
      for num in range(numberOfBikes):
        self.bikesQueue.put("bike")
  def pickUpBike(self):
    return self.bikesQueue.get() ##waits to get a bike if there aren't any available, need to return so that it stops waiting
  def returnBike(self):
    self.bikesQueue.put("bike")
  def addRider(self):
    self.visitors+=1




def bikerTask3(stations,stationAProbabilities, env, meanRate,QMatrix,success, std, mu, stationProbabilities,avgWaitingTime):

  yield env.timeout(random.expovariate(meanRate))
  ##picking a station randomly
  stationChosen= np.random.choice(len(stationAProbabilities), p=stationProbabilities)
  stationName=list(stationAProbabilities.keys())[stationChosen]
  station=stations[stationName] ##now station is the station object
  ##keep track of the number of people at that station
  station.addRider()
  beforePickingUp=env.now
  yield station.pickUpBike() ##waits until a bike is available
  afterPickingUp=env.now
  waitingTime=afterPickingUp-beforePickingUp
  avgWaitingTime.append(waitingTime)
  success[0]+=1
  ##choosing destination randomly
  startArray=QMatrix[stationName]
  ##now startArray is list of tuples with ending station and probabiities
  probabilities=[]
  for end,probability in startArray:
    probabilities.append(probability)

  destinatationStation=np.random.choice(len(startArray), p=probabilities) ##getting probability of going to station j starting at stationChosen index



  destination=startArray[destinatationStation][0]
  yield env.timeout(np.random.lognormal(mu,std)) ##random amount of time that the biker uses the bike

  if destination in stations:

    destinationObject=stations[destination]
    destinationObject.returnBike() ##Returning the bike to the destintations station





In [ ]:
def simulationTask3(numberOfStations,bikesPerStation, numberOfRiders,meanRate,QMatrix,std,mu,stationAProbabilities,adict):
  stations={}
  env=simpy.Environment()
  stationProbabilities=list(stationAProbabilities.values())
  infinite=False
  if(bikesPerStation==float('inf')):
    infinite=True
  for stationName,probability in stationAProbabilities.items():
    stations[stationName]=BikeStationTask3(env,bikesPerStation,stationName,infinite)
  avgWaitingTime=[]
  success=[0]
  for num in range(numberOfRiders):
    env.process(bikerTask3(stations,stationAProbabilities, env, meanRate,QMatrix,success, std, mu, stationProbabilities,avgWaitingTime))
  env.run(until=60*24)  # Run the simulation for 24 hours (in minutes)
  sum=0
  count=0
  for num in avgWaitingTime:
    sum+=num
    count+=1
  avgWaiting=sum/count
  for key,value in stations.items():
    if key not in adict:
      adict[key]=[]
    adict[key].append(value.visitors)

  return success[0],avgWaiting,adict  # Return the count of successful pickups



In [ ]:
num_stations = len(stationAProbabilities)
bikesPerStation=10
num_riders = 3500
lambda_rate = 2.38
q_matrix = Q
std=.619
mu = 2.78
adict={}


for num in range(50):
  successful_rides,waiting,adict = simulationTask3(num_stations,bikesPerStation, num_riders,lambda_rate,q_matrix,std,mu,stationAProbabilities,adict)
print(adict)
print(f"Total successful rides: {successful_rides}")
print(f"Waiting : {waiting}")


{'South Waterfront Walkway - Sinatra Dr & 1 St': [160, 170, 149, 151, 157, 139, 159, 160, 147, 176, 157, 146, 170, 153, 141, 164, 159, 164, 141, 158, 166, 165, 139, 142, 148, 159, 149, 164, 160, 157, 138, 153, 156, 183, 175, 150, 158, 146, 163, 170, 159, 137, 140, 155, 163, 147, 146, 162, 162, 156], 'Grove St PATH': [158, 146, 161, 147, 166, 167, 158, 186, 136, 134, 133, 150, 161, 155, 165, 166, 142, 164, 155, 141, 140, 152, 153, 163, 145, 132, 156, 124, 152, 147, 157, 159, 145, 147, 135, 161, 147, 154, 145, 166, 157, 146, 167, 147, 129, 151, 145, 146, 162, 138], 'Hoboken Terminal - Hudson St & Hudson Pl': [131, 116, 133, 114, 116, 114, 118, 111, 119, 119, 129, 130, 124, 128, 118, 118, 120, 133, 128, 113, 110, 106, 113, 118, 112, 135, 126, 122, 108, 111, 132, 116, 126, 117, 124, 115, 103, 114, 129, 108, 117, 131, 114, 112, 109, 114, 136, 120, 138, 114], 'Hoboken Terminal - River St & Hudson Pl': [120, 106, 97, 92, 121, 108, 117, 83, 123, 111, 95, 101, 106, 91, 110, 98, 114, 107, 110, 1

In [ ]:
import math
stationMaxes={}
for key,value in adict.items():

  stationMaxes[key]=max(value)

In [ ]:
stationMaxes

{'South Waterfront Walkway - Sinatra Dr & 1 St': 183,
 'Grove St PATH': 186,
 'Hoboken Terminal - Hudson St & Hudson Pl': 138,
 'Hoboken Terminal - River St & Hudson Pl': 133,
 'Newport Pkwy': 113,
 'City Hall - Washington St & 1 St': 121,
 'Newport PATH': 107,
 '12 St & Sinatra Dr N': 105,
 'Hoboken Ave at Monmouth St': 98,
 'Marin Light Rail': 98,
 'Hamilton Park': 98,
 '14 St Ferry - 14 St & Shipyard Ln': 94,
 'Liberty Light Rail': 96,
 'Columbus Dr at Exchange Pl': 83,
 'Harborside': 80,
 '11 St & Washington St': 72,
 'Washington St': 81,
 'Sip Ave': 74,
 'Hudson St & 4 St': 80,
 '8 St & Washington St': 76,
 'Madison St & 1 St': 72,
 'City Hall': 65,
 'Warren St': 70,
 'Newark Ave': 69,
 'Columbus Park - Clinton St & 9 St': 63,
 'Grand St & 14 St': 61,
 'Church Sq Park - 5 St & Park Ave': 62,
 'Columbus Drive': 59,
 'Van Vorst Park': 56,
 'Clinton St & Newark St': 63,
 'Grand St': 56,
 'Paulus Hook': 53,
 'Manila & 1st': 56,
 '9 St HBLR - Jackson St & 8 St': 52,
 'Bloomfield St & 1